In [13]:
# Bell State Generator (Self-Contained)
# =====================================
#
# This notebook contains a minimal quantum library (`State`, `Operator`) and
# functions to generate Bell, GHZ, and W states.

import math
import numpy as np

# --- 1. Minimal Quantum Library ---

class State(np.ndarray):
  """Subclassing numpy array to add quantum state methods."""

  def __new__(cls, input_array):
    obj = np.asarray(input_array).view(cls)
    return obj

  @property
  def nbits(self):
    return int(math.log2(self.shape[0]))

  def is_close(self, other):
    return np.allclose(self, other)

  def density(self):
    """Return density matrix |psi><psi|."""
    mat = np.outer(self, np.conj(self))
    return Operator(mat)
  
  def prob(self, *bits):
    """Return probability of a specific bitstring."""
    idx = 0
    for i, bit in enumerate(bits):
      idx = (idx << 1) | bit
    return np.real(np.abs(self[idx])**2)
    
  def is_pure(self):
    return True

class Operator(np.ndarray):
  """Subclassing numpy array for operators."""

  def __new__(cls, input_array):
    obj = np.asarray(input_array).view(cls)
    return obj

  def __array_finalize__(self, obj):
    if obj is None: return

  @property
  def nbits(self):
    return int(np.log2(self.shape[0]))

  def __mul__(self, other):
    """Tensor product: self * other -> self (x) other."""
    if isinstance(other, Operator):
        return Operator(np.kron(self, other))
    return Operator(np.multiply(self, other))

  def __call__(self, s, idx=0):
    """Apply operator to state s: Op(s)."""
    if isinstance(s, State):
      if self.nbits < s.nbits:
         # Construct Full Operator: Prefix (I) @ Op @ Suffix (I)
         prefix = Identity(idx) if idx > 0 else None
         suffix_bits = s.nbits - idx - self.nbits
         suffix = Identity(suffix_bits) if suffix_bits > 0 else None
         
         op = self
         if prefix is not None:
             op = np.kron(prefix, op)
         if suffix is not None:
             op = np.kron(op, suffix)
         return State(op @ s)
    return State(self @ s)

  def is_pure(self):
    """Check if density matrix is pure: tr(rho^2) == 1."""
    return np.isclose(np.real(np.trace(self @ self)), 1.0)

# --- Helpers & Gates ---

def zeros(n):
  s = np.zeros(2**n, dtype=np.complex128)
  s[0] = 1.0
  return State(s)

def ones(n):
  s = np.zeros(2**n, dtype=np.complex128)
  s[(2**n)-1] = 1.0
  return State(s)

def bitstring(*bits):
  idx = 0
  for bit in bits:
    idx = (idx << 1) | bit
  s = np.zeros(2**len(bits), dtype=np.complex128)
  s[idx] = 1.0
  return State(s)

def Identity(n=1):
  dim = 2**n
  return Operator(np.eye(dim, dtype=np.complex128))

def Hadamard():
  h = (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]], dtype=np.complex128)
  return Operator(h)

def PauliX():
  return Operator(np.array([[0, 1], [1, 0]], dtype=np.complex128))

def PauliY():
  return Operator(np.array([[0, -1j], [1j, 0]], dtype=np.complex128))

def PauliZ():
  return Operator(np.array([[1, 0], [0, -1]], dtype=np.complex128))

def Cnot(control=0, target=1):
  if control == 0 and target == 1:
      mat = np.array([
          [1, 0, 0, 0],
          [0, 1, 0, 0],
          [0, 0, 0, 1],
          [0, 0, 1, 0]
      ], dtype=np.complex128)
      return Operator(mat)
  raise NotImplementedError("Only Cnot(0,1) implemented")

def Measure(psi, idx, tostate=0, collapse=True):
  """Measure qubit idx."""
  nbits = psi.nbits
  
  # Projector selection
  if tostate == 0:
      P = np.array([[1, 0], [0, 0]], dtype=np.complex128)
  else:
      P = np.array([[0, 0], [0, 1]], dtype=np.complex128)
      
  # Expand Projector
  Op = P if idx == 0 else np.eye(2)
  for i in range(1, nbits):
      if i == idx:
          Op = np.kron(Op, P)
      else:
          Op = np.kron(Op, np.eye(2))
          
  projected = Op @ psi
  prob = np.real(np.vdot(psi, projected))
  
  if not collapse:
      return prob, psi

  if prob < 1e-9:
       return 0.0, psi
       
  new_psi = State(projected / np.sqrt(prob))
  return prob, new_psi


In [14]:
# --- 2. State Generators ---

def bell_state(a, b):
    # Start with |a, b>
    psi = bitstring(a, b)
    
    # H on qubit 0 -> (H x I)
    H_I = Operator(np.kron(Hadamard(), Identity()))
    psi = State(H_I @ psi)
    
    # CNOT(0, 1)
    C = Cnot(0, 1)
    psi = State(C @ psi)
    return psi

def ghz_state(n):
    # |0...0> + |1...1>
    s0 = zeros(n)
    s1 = ones(n)
    return State((s0 + s1) / np.sqrt(2))

def w_state():
    # |W> = 1/sqrt(3) (|001> + |010> + |100>)
    arr = np.zeros(8, dtype=np.complex128)
    arr[1] = 1/np.sqrt(3)  # 001
    arr[2] = 1/np.sqrt(3)  # 010
    arr[4] = 1/np.sqrt(3)  # 100
    return State(arr)


In [17]:
# --- 3. Demonstration ---

print(f"{'Bell State':<12} | {'Input (a,b)':<12} | {'State Vector'}")
print("-" * 60)
for (a, b), name in zip([(0,0), (0,1), (1,0), (1,1)], ["Phi+", "Psi+", "Phi-", "Psi-"]):
    psi = bell_state(a, b)
    print(f"{name:<12} | {str((a,b)):<12} | {np.round(psi, 3).tolist()}")

print("\nGHZ(3):\n", np.round(ghz_state(3), 3).tolist())
print("\nW(3):\n", np.round(w_state(), 3).tolist())


Bell State   | Input (a,b)  | State Vector
------------------------------------------------------------
Phi+         | (0, 0)       | [(0.707+0j), 0j, 0j, (0.707+0j)]
Psi+         | (0, 1)       | [0j, (0.707+0j), (0.707+0j), 0j]
Phi-         | (1, 0)       | [(0.707+0j), 0j, 0j, (-0.707+0j)]
Psi-         | (1, 1)       | [0j, (0.707+0j), (-0.707+0j), 0j]

GHZ(3):
 [(0.707+0j), 0j, 0j, 0j, 0j, 0j, 0j, (0.707+0j)]

W(3):
 [0j, (0.577+0j), (0.577+0j), 0j, (0.577+0j), 0j, 0j, 0j]
